# Tracking Bugs

So far, we have assumed that failures would be discovered and fixed by a single programmer during development. But what if the user who discovers a bug is different from the developer who eventually fixes it? In this case, users have to _report_ bugs, and one needs to ensure that reported bugs are systematically _tracked_. This is the job of dedicated _bug tracking systems_, which we will discuss (and demo) in this chapter.

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("bJzHYzvxHm8")

**Prerequisites**

* You should have read the [Introduction to Debugging](Intro_Debugging.ipynb).

In [ ]:
import bookutils

In [ ]:
import Intro_Debugging

In [ ]:
import os
import sys

In [ ]:
# ignore
if 'CI' in os.environ:
    # Can't run this in our continuous environment,
    # since it can't run a headless Web browser
    sys.exit(0)

In [ ]:
# ignore
#
# WARNING: Unlike the other chapters in this book, 
# this chapter should NOT BE RUN AS A NOTEBOOK:
#
# * It will delete ALL data from an existing 
#   local _Redmine_ installation.
# * It will create new users and databases in an existing
#   local _MySQL_ installation.
# 
# The only reason to run this notebook is to create the book chapter,
# which is the task of Andreas Zeller (and possibly some translators).
# If you are not Andreas, you should exactly know what you are doing.

assert os.getenv('USER') == 'zeller'

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from debuggingbook.Tracking import <identifier>
```

and then make use of the following features.


This chapter provides no functionality that could be used by third-party code.



## Reporting Issues

So far, we have always assumed an environment in which software failures would be discovered by the very developers responsible for the code – that is, failures discovered (and fixed) during development. However, failures can also be discovered by third parties, such as

* _Testers_ whose job it is to test the code of developers
* Other _developers_ using the code
* _Users_ running the code as it is in production

In all these cases, developers need to be _informed_ about the fact that the program failed; if they won't know that a bug exists, it will be hard to fix it. This means that we have to set up mechanisms for _reporting bugs_ – manual ones and/or automated ones.

### What Goes in a Bug Report?

Let us start with the information a developer requires to _fix_ a bug. In a 2008 study \cite{Bettenburg2008}, Bettenburg et al. asked 872 developers from the Apache, Eclipse, and Mozilla projects to complete a survey on the most important information they need. From top to bottom, these were as follows:

#### Steps to Reproduce (83%)

This is a list of steps by which the failure would be reproduced. For instance:
> 1. I started the program using `$ python Debugger.py my_code.py`.
> 2. Then, at the `(debugger)` prompt, I entered `run` and pressed the <kbd>ENTER</kbd> key.

The easier it will be for the developer to reproduce the bug, the higher the chances it will be effectively fixed. [Reducing the steps to those relevant to reproduce the bug](DeltaDebugger.ipynb) can be helpful. But at the same time, the main problem experienced by developers as it comes to bug reports is _incomplete information_, and this especially applies to the steps to reproduce.

#### Stack Traces (57%)

These give hints on which parts of the code were active at the moment the failure occurred.
> I got this stack trace:
```python
Traceback (most recent call last):
  File "Debugger.py", line 2, in <module>
    handle_command("run")
  File "Debugger.py", line 3, in handle_command
    scope = s.index(" in ")
ValueError: substring not found (expected)
```

Even though stack traces are useful, they are seldom reported by regular users, as they are difficult to obtain (or to find if included in log files). Automated crash reports (see below), however, frequently include them.

#### Test Cases (51%)

Test cases that reproduce the bug are also seen as important:
> I can reproduce the bug using the following code:
```python
import Debugger

Debugger.handle_command("run")
```

Non-developers hardly ever report test cases.

#### Observed Behavior (33%)

What the bug reporter observed as a failure.
> The program crashed with a `ValueError`.

In many cases, this mimics the stack trace or the steps to reproduce the bug.

#### Screenshots (26%)

Screenshots can further illustrate the failure.
> Here is a screenshot of the `Debugger` failing in Jupyter.

Screenshots are helpful for certain bugs, such as GUI errors.

#### Expected Behavior (22%)

What the bug reporter expected instead.
> I expected the program not to crash.

#### Configuration Information (< 12%)

Perhaps surprisingly, the information that was seen as _least_ relevant for developers was:

* Version (12%)
* Build information (8%)
* Product (5%)
* Operating system (4%)
* Component (3%)
* Hardware (0%)

The relative low importance of these fields may be surprising as entering them is usually mandated in bug report forms. However, in \cite{Bettenburg2008}, developers stated that 
> [Operating System] fields are rarely needed as most [of] our bugs are usually found on all platforms.

This not meant to be read as these fields being totally irrelevant, as, of course, there can be bugs that occur only on specific platforms. Also, if a bug is reported for an older version, but is known to be fixed in a more current version, a simple resolution would be to ask the user to upgrade to the fixed version.

### Reporting Crashes Automatically

If a program crashes, it can be a good idea to have it automatically report the failure to developers. A common practice is to have a crashing program show a _bug report dialog_ allowing the user to report the crash to the vendor. The user is typically asked to provide additional details on how the failure came to be, and the crash report would be sent directly to the vendor's database:

![](https://upload.wikimedia.org/wikipedia/commons/b/b0/Screenshot-Bug_Buddy.png)

The automatic report typically includes a _stack trace_ and _configuration information_. These two do not reveal too many sensitive details about the user, yet already can help a lot in fixing a bug. In the interest of transparency, the user should be able to inspect all information sent to the vendor.

Besides stack traces and configuration information, such _crash reporters_ could, of course, collect much more - say the data the program operated on, logs, recorded steps to reproduce, or automatically recorded screenshots. However, all of these will likely include sensitive information; and despite their potential usefulness, it is typically better to not collect them in the first place.

### Effective Issue Reporting

When writing an issue report, it is important to look at it from _the developer's perspective_. Developers not only require information on how to reproduce the bug; they also want to work _effectively_ and _efficiently_. The following aspects will all help developers with that:

* __Have a concise summary__. A good summary should _quickly_ and _uniquely_ identify a bug. Make it easy to understand such that the reader can know if the bug has been reported and/or fixed.
* __Be clear and concise__. Provide necessary information (as shown above) and avoid any extras. Use meaningful sentences and simple words. Structure the report into enumerations and bullet lists.
* __Do not assume context__. Make no assumption that the developer knows all about your  bug. Find out whether similar issues have been reported before and reference them.
* __Avoid commanding tones__. Developers enjoy autonomy in their work, and coming across as too authoritative hurts morale.
* __Avoid sarcasm__. Same as above. If you think you can get a volunteer to fix a bug in an open source program for you with sarcasm or commands – good luck with that.
* __Do not assume mistakes__. Do not assume some developer (or anyone) has made a mistake. In many cases, issues can be resolved on your side.
* __One issue per report__. If you have multiple issues, split them in multiple reports; this makes it easier to process them.

## An Issue Tracker

At the developer's end, all issues reported need to be _tracked_ – that is, they have to be _registered_, they have to be _checked_, and of course, they have to be _addressed_. This process takes place via dedicated database systems, so-called _bug tracking systems_. 

The purposes of an issue tracking system include

* to collect and store all issue reports;
* to check the status of issues at all times; and
* to organize the debugging and development process.

Let us illustrate how these steps work, using the popular `Redmine` issue tracking system.

### Excursion: Setting up Redmine

To install Redmine, we followed the instructions at https://gist.github.com/johnjohndoe/2763243.
These final steps initialize the database:

In [ ]:
import subprocess

In [ ]:
import os
import sys

In [ ]:
def with_ruby(cmd: str, inp: str = '', timeout: int = 30, show_stdout: bool = False) -> None:
    print(f"$ {cmd}")
    shell = subprocess.Popen(['/bin/sh', '-c',
        f'''rvm_redmine=$HOME/.rvm/gems/ruby-2.7.2@redmine; \
rvm_global=$HOME/.rvm/gems/ruby-2.7.2@global; \
export GEM_PATH=$rvm_redmine:$rvm_global; \
export PATH=$rvm_redmine/bin:$rvm_global/bin:$HOME/.rvm/rubies/ruby-2.7.2/bin:$HOME/.rvm/bin:$PATH; \
cd $HOME/lib/redmine && {cmd}'''],
                             stdin=subprocess.PIPE,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE,
                             universal_newlines=True)
    try:
        stdout_data, stderr_data = shell.communicate(inp, timeout=timeout)
    except subprocess.TimeoutExpired:
        shell.kill()
#         stdout_data, stderr_data = shell.communicate(inp)
#         if show_stdout:
#             print(stdout_data, end="")
#         print(stderr_data, file=sys.stderr, end="")
        raise

    print(stderr_data, file=sys.stderr, end="")
    if show_stdout:
        print(stdout_data, end="")

In [ ]:
def with_mysql(cmd: str, timeout: int = 2, show_stdout: bool = False) -> None:
    print(f"sql>{cmd}")
    sql = subprocess.Popen(["mysql", "-u", "root",
                           "--default-character-set=utf8mb4"],
                            stdin=subprocess.PIPE,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE, 
                            universal_newlines=True)
    try:
        stdout_data, stderr_data = sql.communicate(cmd + ';', 
                                                   timeout=timeout)
    except subprocess.TimeoutExpired:
        sql.kill()
#         stdout_data, stderr_data = sql.communicate(inp)
#         if show_stdout:
#             print(stdout_data, end="")
#         print(stderr_data, file=sys.stderr, end="")
        raise

    print(stderr_data, file=sys.stderr, end="")
    if show_stdout:
        print(stdout_data, end="")

In [ ]:
with_ruby("bundle config set without development test")

In [ ]:
with_ruby("bundle install")

In [ ]:
with_ruby("pkill sql; sleep 5")

In [ ]:
try:
    with_ruby("mysql.server start", show_stdout=True)
except subprocess.TimeoutExpired:
    pass  # Can actually start without producing output

In [ ]:
with_mysql("drop database redmine")

In [ ]:
with_mysql("drop user 'redmine'@'localhost'")

In [ ]:
with_mysql("create database redmine character set utf8")

In [ ]:
with_mysql("create user 'redmine'@'localhost' identified by 'my_password'")

In [ ]:
with_mysql("grant all privileges on redmine.* to 'redmine'@'localhost'")

In [ ]:
with_ruby("bundle exec rake generate_secret_token")

In [ ]:
with_ruby("RAILS_ENV=production bundle exec rake db:migrate")

In [ ]:
with_ruby("RAILS_ENV=production bundle exec rake redmine:load_default_data", '\n')

### End of Excursion

### Excursion: Starting Redmine

In [ ]:
import os
import time

In [ ]:
from multiprocess import Process  # type: ignore

In [ ]:
# ignore
from typing import Tuple

In [ ]:
def run_redmine(port: int) -> None:
    with_ruby(f'exec rails s -e production -p {port} > redmine.log 2>&1',
              timeout=3600)

In [ ]:
def start_redmine(port: int = 3000) -> Tuple[Process, str]:
    process = Process(target=run_redmine, args=(port,))
    process.start()
    time.sleep(5)

    url = f"http://localhost:{port}"
    return process, url

In [ ]:
redmine_process, redmine_url = start_redmine()

### End of Excursion

### Excursion: Remote Control with Selenium

To produce this book, we use Selenium to interact with user interfaces and obtain screenshots.

[Selenium](https://www.seleniumhq.org) is a framework for testing Web applications by _automating interaction in the browser_.  Selenium provides an API that allows one to launch a Web browser, query the state of the user interface, and interact with individual user interface elements.  The Selenium API is available in a number of languages; we use the [Selenium API for Python](https://selenium-python.readthedocs.io/index.html).

A Selenium *web driver* is the interface between a program and a browser controlled by the program.

In [ ]:
from selenium import webdriver

In [ ]:
from selenium.webdriver.common.keys import Keys

The following code starts a Firefox browser in the background, which we then control through the web driver.

In [ ]:
BROWSER = 'firefox'

In [ ]:
with_ruby("pkill Firefox.app firefox-bin")

**Note:** If you don't have Firefox installed, you can also set `BROWSER` to `'chrome'` to use Google Chrome instead.

In [ ]:
# BROWSER = 'chrome'

When running this outside of Jupyter notebooks, the browser is _headless_, meaning that it does not show on the screen.

In [ ]:
from bookutils import rich_output

In [ ]:
# HEADLESS = not rich_output()
HEADLESS = True

In [ ]:
from selenium.webdriver.remote.webdriver import WebDriver

In [ ]:
def start_webdriver(browser: str = BROWSER, headless: bool = HEADLESS, zoom: float = 4.0) -> WebDriver:
    if browser == 'firefox':
        options = webdriver.FirefoxOptions()
    if browser == 'chrome':
        options = webdriver.ChromeOptions()

    if headless and browser == 'chrome':
        options.add_argument('headless')
    else:
        options.headless = headless

    # Start the browser, and obtain a _web driver_ object such that we can interact with it.
    if browser == 'firefox':
        # For firefox, set a higher resolution for our screenshots
        profile = webdriver.firefox.firefox_profile.FirefoxProfile()
        profile.set_preference("layout.css.devPixelsPerPx", repr(zoom))
        redmine_gui = webdriver.Firefox(firefox_profile=profile, 
                                        options=options)  # type: ignore

        # We set the window size such that it fits
        redmine_gui.set_window_size(500, 600)  # was 1024, 600

    elif browser == 'chrome':
        redmine_gui = webdriver.Chrome(options=options)  # type: ignore
        redmine_gui.set_window_size(1024, 510 if headless else 640)

    return redmine_gui

In [ ]:
redmine_gui = start_webdriver(browser=BROWSER, headless=HEADLESS)

We can now interact with the browser programmatically.  First, we have it navigate to the URL of our Web server:

In [ ]:
redmine_gui.get(redmine_url)

To see what the "headless" browser displays, we can obtain a screenshot.  We see that it actually displays the home page.

In [ ]:
from IPython.display import display, Image

In [ ]:
Image(redmine_gui.get_screenshot_as_png())

### End of Excursion

### Excursion: Screenshots with Drop Shadows

By default, our screenshots are flat. We add a drop shadow to make them look nicer.
With help from https://graphicdesign.stackexchange.com/questions/117272/how-to-add-drop-shadow-to-a-picture-via-cli

In [ ]:
import tempfile

In [ ]:
def drop_shadow(contents: bytes) -> bytes:
    with tempfile.NamedTemporaryFile() as tmp:
        tmp.write(contents)
        convert = subprocess.Popen(
            ['convert', tmp.name,
            '(', '+clone', '-background', 'black', '-shadow', '50x10+15+15', ')',
            '+swap', '-background', 'none', '-layers', 'merge', '+repage', '-'],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout_data, stderr_data = convert.communicate()

    if stderr_data:
        print(stderr_data.decode("utf-8"), file=sys.stderr, end="")

    return stdout_data

In [ ]:
def screenshot(driver: WebDriver, width: int = 500) -> bytes:
    return Image(drop_shadow(redmine_gui.get_screenshot_as_png()), width=width)

In [ ]:
screenshot(redmine_gui)

### End of Excursion

### Excursion: First Registration at Redmine

In [ ]:
redmine_gui.get(redmine_url + '/login')

In [ ]:
screenshot(redmine_gui)

In [ ]:
redmine_gui.find_element_by_id("username").send_keys("admin")
redmine_gui.find_element_by_id("password").send_keys("admin")
redmine_gui.find_element_by_name("login").click()

In [ ]:
time.sleep(2)

In [ ]:
if redmine_gui.current_url.endswith('my/password'):
    redmine_gui.get(redmine_url + '/my/password')
    redmine_gui.find_element_by_id("password").send_keys("admin")
    redmine_gui.find_element_by_id("new_password").send_keys("admin001")
    redmine_gui.find_element_by_id("new_password_confirmation").send_keys("admin001")
    display(screenshot(redmine_gui))
    redmine_gui.find_element_by_name("commit").click()

In [ ]:
redmine_gui.get(redmine_url + '/logout')
redmine_gui.find_element_by_name("commit").click()

### End of Excursion

This is what the _Redmine_ tracker starts with:

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/login')
screenshot(redmine_gui)

After we login, we see our account:

In [ ]:
# ignore
redmine_gui.find_element_by_id("username").send_keys("admin")
redmine_gui.find_element_by_id("password").send_keys("admin001")
redmine_gui.find_element_by_name("login").click()
screenshot(redmine_gui)

### Excursion: Creating a Project

Let us check out the projects by clicking on "Projects".

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/projects')
screenshot(redmine_gui)

We can now enter a project name and a description.

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/projects/new')
screenshot(redmine_gui)

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/projects/new')
redmine_gui.find_element_by_id('project_name').send_keys("The Debugging Book")
redmine_gui.find_element_by_id('project_description').send_keys("A Book on Automated Debugging")
redmine_gui.find_element_by_id('project_identifier').clear()
redmine_gui.find_element_by_id('project_identifier').send_keys("debuggingbook")
redmine_gui.find_element_by_id('project_homepage').send_keys("https://www.debuggingbook.org/")
screenshot(redmine_gui)

In [ ]:
# ignore
redmine_gui.find_element_by_name('commit').click()

### End of Excursion

We start with a list of projects.

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/projects')
screenshot(redmine_gui)

Let us choose the (one) "debuggingbook" project.

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/projects/debuggingbook')
screenshot(redmine_gui)

## Reporting an Issue

The most basic task of a bug tracker is to _report_ bugs. However, a bug tracker is a bit more general than just bugs – it can also track feature requests, support requests, and more. These are all summarized under the term "issue".

Let us take the role of a bug reporter – pardon, an _issue_ reporter – and report an issue. We can do this right from the _Redmine_ menu.

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/issues/new')
screenshot(redmine_gui)

Let's give our bug a name:

In [ ]:
issue_title = "Does not render correctly on Nokia Communicator"

In [ ]:
issue_description = \
"""The Debugging Book does not render correctly on the Nokia Communicator 9000.

Steps to reproduce:
1. On the Nokia, go to "https://debuggingbook.org/"
2. From the menu on top, select the chapter "Tracking Origins".
3. Scroll down to a place where a graph is supposed to be shown.
4. Instead of the graph, only a blank space is displayed.

How to fix:
* The graphs seem to come as SVG elements, but the Nokia Communicator does not support SVG rendering. Render them as JPEGs instead.
"""

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/issues/new')

redmine_gui.find_element_by_id('issue_subject').send_keys(issue_title)
redmine_gui.find_element_by_id('issue_description').send_keys(issue_description)
screenshot(redmine_gui)

In [ ]:
# ignore
redmine_gui.find_element_by_id('issue_assigned_to_id').click()
screenshot(redmine_gui)

In [ ]:
# ignore
redmine_gui.execute_script("window.scrollTo(0, document.body.scrollHeight);")
screenshot(redmine_gui)

Clicking on "Create" creates the new issue report.

In [ ]:
# ignore
redmine_gui.find_element_by_name('commit').click()
screenshot(redmine_gui)

Our bug has been assigned an _issue number_ (#1 in that case). This issue number (also known as bug number, or problem report number) will be used to identify the specific issue from this point on in all communication between developers. Developers will know which issues they are working on; and in any change to the software, they will refer to the issue the change relates to.

Over time, issue databases can grow massively, especially for popular products with several users. As an example, consider the [Mozilla _Bugzilla_ issue tracker](https://bugzilla.mozilla.org/), collecting issue reports for the Firefox browser and other Mozilla products.

In [ ]:
# ignore
from bookutils import quiz

In [ ]:
quiz("How many issues have been reported over time in Mozilla Bugzilla?",
    [
        "More than ten thousand",
        "More than a hundred thousand",
        "More than a million",
        "More than ten million"
    ], '370370367 // 123456789')

Yes, it is that many! Look at the bug IDs in this recent screenshot from the Mozilla Bugzilla database:

In [ ]:
# ignore
redmine_gui.get("https://bugzilla.mozilla.org/buglist.cgi?quicksearch=firefox")

In [ ]:
# ignore
screenshot(redmine_gui)

Let us enter a few more reports into our issue tracker.

### Excursion: Adding Some More Issue Reports

In [ ]:
def new_issue(issue_title: str, issue_description: str) -> bytes:
    redmine_gui.get(redmine_url + '/issues/new')

    redmine_gui.find_element_by_id('issue_subject').send_keys(issue_title)
    redmine_gui.find_element_by_id('issue_description').send_keys(issue_description)
    redmine_gui.find_element_by_name('commit').click()
    return screenshot(redmine_gui)

In [ ]:
new_issue("Missing a Chapter on Parallel Debugging",
          """I am missing a chapter on (automatic) debugging of parallel and distributed systems,
including how to detect and repair data races, log message passing, and more.
In my experience, almost all programs are parallel today, so you are missing
an important subject.
""")

In [ ]:
new_issue("Missing a PDF version",
          """Your 'book' does not provide a printed version. I think that printed books

* offer a more enjoyable experience for the reader
* allow me to annotate pages with my own remarks
* allow me to set dog-ear bookmatks
* allow me to show off what I'm currently reading (do you have a cover, too?)

Please provide a printed version - or, at least, produce a PDF version
of the debugging book, and make it available for download, such that I can print it myself.
""")

In [ ]:
new_issue("No PDF version",
          """Can I have a printed version of your book? Please!""")

In [ ]:
new_issue("Does not work with Python 2.7 or earlier",
          """I was deeply disappointed that your hew book requires Python 3.9 or later.
There are still several Python 2.x users out here (I, for one, cannot stand having to
type parentheses for every `print` statement), and I would love to run your code on
my Python 2.7 programs.

Would it be possible to backport the book's code such that it would run on Python 3.x
as well as Python 2.x? I would suggest that you add simple checks around your code
such as the following:

```
import sys

if sys.version_info.major >= 3:
    print("The result is", x)
else: 
    print "The result is", x
```

As an alternative, rewrite the book in Python 2 and have it automatically translate to
Python 3. This way, you could address all Python lovers, not just Python 3 ones.
""")

In [ ]:
new_issue("Support for C++",
          """I had lots of fun with your 'debugging book'. Yet, I was somewhat disappointed
to see that all code examples are in and for Python programs only. Is there a chance
to get them to work on a real programming language such as C or C++? This would also
open the way to discuss several new debugging techniques for bugs that occur in these
languages only. A chapter on C++ move semantics, and how to fix them, for instance,
would be highly appreciated.
""")

### End of Excursion

## Managing Issues

Let us now switch sides and take the view of a _developer_ whose job it is to actually _handle_ all these issues. When our developers log in, the first thing they see is that there are a number of new issues that are all "open" – that is, in need to be addressed. (For a typical developer, this may well be the first task of the day.)

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/projects/debuggingbook")
screenshot(redmine_gui)

Clicking on "View all issues" shows us all issues reported so far.

In [ ]:
# ignore
redmine_gui.get(redmine_url + '/projects/debuggingbook/issues')
redmine_gui.execute_script("window.scrollTo(0, document.body.scrollHeight);")
screenshot(redmine_gui)

Let us do some bug _triaging_ here. Bug report #2 is not a bug – it is a feature request. We invoke its "actions" pop-up menu and mark it as "Feature".

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/issues/")

In [ ]:
# ignore
redmine_gui.find_element_by_xpath("//tr[@id='issue-2']//a[@title='Actions']").click()
time.sleep(0.25)

In [ ]:
# ignore
tracker_item = redmine_gui.find_element_by_xpath(
    "//div[@id='context-menu']//a[text()='Tracker']")
actions = webdriver.ActionChains(redmine_gui)
actions.move_to_element(tracker_item)
actions.perform()
screenshot(redmine_gui)

In [ ]:
# ignore
redmine_gui.find_element_by_xpath("//div[@id='context-menu']//a[text()='Feature']").click()

The same applies to bugs #3 and #4 (missing PDF) and #6 (no support for C++). We mark them as such as well.

In [ ]:
# ignore
def mark_tracker(issue: int, tracker: str) -> None:
    redmine_gui.get(redmine_url + "/issues/")
    redmine_gui.find_element_by_xpath(
        f"//tr[@id='issue-{str(issue)}']//a[@title='Actions']").click()
    time.sleep(0.25)

    tracker_item = redmine_gui.find_element_by_xpath(
        "//div[@id='context-menu']//a[text()='Tracker']")
    actions = webdriver.ActionChains(redmine_gui)
    actions.move_to_element(tracker_item)
    actions.perform()
    time.sleep(0.25)

    redmine_gui.find_element_by_xpath(
        f"//div[@id='context-menu']//a[text()='{tracker}']").click()

In [ ]:
# ignore
mark_tracker(3, "Feature")
mark_tracker(4, "Feature")
mark_tracker(6, "Feature")

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/issues/")
redmine_gui.execute_script("window.scrollTo(0, document.body.scrollHeight);")
screenshot(redmine_gui)

The fact that we marked these issues as "feature requests" does not mean that they will not be worked on – on the contrary, a feature requested by management can have an even higher priority than a bug. Right now, though, we will give our first priority to the bugs listed.

## Assigning Priorities

After we have decided the priority for individual bugs, we can make this decision explicit by assigning a _priority_ to each bug. This allows our co-developers to see which things are the most pressing to work on.

Let us assume we have an important customer for whom fixing issue #1 is important. Through the context menu, we can assign issue #1 a priority of "Urgent". 

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/issues/")

In [ ]:
# ignore
redmine_gui.find_element_by_xpath("//tr[@id='issue-1']//a[@title='Actions']").click()
time.sleep(0.25)

In [ ]:
# ignore
priority_item = redmine_gui.find_element_by_xpath("//div[@id='context-menu']//a[text()='Priority']")
actions = webdriver.ActionChains(redmine_gui)
actions.move_to_element(priority_item)
actions.perform()
screenshot(redmine_gui)

In [ ]:
# ignore
redmine_gui.find_element_by_xpath("//div[@id='context-menu']//a[text()='Urgent']").click()

We see that issue #1 is now listed as urgent.

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/issues/")
redmine_gui.execute_script("window.scrollTo(0, document.body.scrollHeight);")
screenshot(redmine_gui)

On top of priority, some issue trackers also allow to assign a _severity_ to an issue, describing the impact of the bug:

* __Blocker__ (also known as __Showstopper__): Blocks development and/or testing, for instance by breaking build processes.
* __Critical__: Application crash. Loss of data.
* __Major__: Loss of function.
* __Minor__: Incomplete function.
* __Trivial__: Minor issues in user interfaces and documentation.
* __Enhancement__: Request of a new feature or an improvement in an existing one.

Such severities would be important factors in determining the priority of an issue.

## Assigning Issues

So far, all the listed bugs are "unassigned", which means that there is no developer who is currently working on them. Let us assign the "urgent" issue #1 to ourselves, such that we have something to do.

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/issues/")

In [ ]:
# ignore
redmine_gui.find_element_by_xpath("//tr[@id='issue-1']//a[@title='Actions']").click()
time.sleep(0.25)

In [ ]:
# ignore
assignee_item = redmine_gui.find_element_by_xpath(
    "//div[@id='context-menu']//a[text()='Assignee']")
actions = webdriver.ActionChains(redmine_gui)
actions.move_to_element(assignee_item)
actions.perform()
screenshot(redmine_gui)

By choosing the Actions menu, clicking on `Assignee`, and then `<< me >>`, we can assign the issue to ourselves.

In [ ]:
# ignore
redmine_gui.find_element_by_xpath("//div[@id='context-menu']//a[text()='<< me >>']").click()
screenshot(redmine_gui)

Of course, we could also go and assign the issue to other developers – depending on their competence and current workload. Very clearly, the ability to assign issues to individuals is a feature for managers.

## Resolving Issues

Let us switch perspectives again, and now take a _developer's role_ – that is, we now work on the issues assigned to us. We get these by clicking on "Issues assigned to me":

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/projects/debuggingbook/issues?query_id=1")
screenshot(redmine_gui)

By clicking on the issue number, we can see all details about the issue.

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/issues/1")
screenshot(redmine_gui)

We can inspect all features of the issue, including its history.

In [ ]:
# ignore
redmine_gui.execute_script("window.scrollTo(0, document.body.scrollHeight);")
screenshot(redmine_gui)

Let us assume that we do have everything in hand we need to fix the bug. (Including a Nokia Communicator device for testing, that is.) This will take some time, during which we won't need our bug database.

![](https://upload.wikimedia.org/wikipedia/commons/2/2f/Prague_Astronomical_Clock_animated.gif)

Let us now assume that we actually have fixed the bug. This means that we have to go back to our bug database such that we can mark the bug as _resolved_. For this, we open the issue again and click on "Edit", and then change the status from "New" to "Resolved".

In [ ]:
# ignore
redmine_gui.get(redmine_url + "/issues/1/edit")
redmine_gui.find_element_by_id("issue_status_id").click()

In [ ]:
# ignore
redmine_gui.find_element_by_xpath("//option[text()='Resolved']").click()
screenshot(redmine_gui)

At the bottom, we can add more notes.

In [ ]:
# ignore
redmine_gui.execute_script("window.scrollTo(0, document.body.scrollHeight);")
issue_notes = redmine_gui.find_element_by_id("issue_notes")
issue_notes.send_keys("Will only work for Nokia Communicator Rev B and later; "
    "Rev A is still unsupported")
screenshot(redmine_gui)

After clicking on "Submit", we are done. Time for the next bug to fix!

In [ ]:
# ignore
redmine_gui.find_element_by_name("commit").click()
screenshot(redmine_gui)

## The Life Cycle of an Issue

We have successfully reported an issue (as some third party), assigned it (as a manager), and resolved it (as a developer). Great! However, the steps we have been going through are just one way an issue might be handled.

### Resolutions

First of all, an issue can have multiple _resolutions_ besides being fixed. Typical resolutions include the following:

#### FIXED

A fix for this issue is made and committed. This is the resolution we have seen for the "Nokia Communicator" bug #1, above.

#### INVALID

The problem described is not an issue. This is what happens to entries in the issue database that actually do not describe an issue, or that do not provide sufficient information to address it.

#### WONTFIX

The issue described is indeed an issue, but will never be fixed. This is a decision of which features (and fixes!) are part of the product, and which ones are not. Our listing of hypothetic issues, above, has an entry complaining that the book "does not work with Python 2.7 or earlier". This will not be fixed.

#### DUPLICATE

The issue is a duplicate of an existing issue. In our listing of hypothetic issues, we have two issues reporting problems with PDF exports. By making one of these a duplicate of the other. we can ensure that both will be fixed (and checked) at the same time.

If your product has several users, your issue database will have several duplicates, as multiple users will stumble across the same bugs. Some developers consider such duplicates as a burden, as identifying duplicates takes time. However, the study by Bettenburg et al.~\cite{Bettenburg2008} points out that such duplicates have their value, too. One developer is quoted as

> Duplicates are not really problems. They often add useful information. That this information were filed under a new report is not ideal thought.

#### WORKSFORME

The developer could not reproduce the issue, and the code provided no hints on why such a behavior might occur. Such issues are often reopened when more information becomes available.

### An Issue Life Cycle

With this in mind, we can sketch the individual states an issue report goes through: After submission (NEW), the report is assigned to a developer (ASSIGNED), and then resolved (RESOLVED) with one of the resolutions listed above.

In [ ]:
# ignore
from Intro_Debugging import graph  # minor dependency

In [ ]:
# ignore
from IPython.display import display

In [ ]:
# ignore
life_cycle = graph()
life_cycle.attr(rankdir='TB')

life_cycle.node('New', label="<<b>NEW</b>>", penwidth='2.0')
life_cycle.node('Assigned', label="<<b>ASSIGNED</b>>")

with life_cycle.subgraph() as res:
    res.attr(rank='same')
    res.node('Resolved', label="<<b>RESOLVED</b>>", penwidth='2.0')
    res.node('Resolution',
                shape='plain',
                fillcolor='white',
                label="""<<b>Resolution:</b> One of<br align="left"/>
• FIXED<br align="left"/>
• INVALID<br align="left"/>
• DUPLICATE<br align="left"/>
• WONTFIX<br align="left"/>
• WORKSFORME<br align="left"/>
>""")
    res.node('Reopened', label="<<b>REOPENED</b>>", style='invis')

life_cycle.edge('New', 'Assigned', label=r"Assigned\lto developer")
life_cycle.edge('Assigned', 'Resolved', label="Developer has fixed bug")

life_cycle.edge('Resolution', 'Resolved', arrowhead='none', style='dashed')

life_cycle

Unfortunately, software development is more complicated than that. There are many things that can happen while an issue is being worked on.

* When an issue report comes in, it may be resolved as "invalid" or "duplicate" before even being assigned to a developer.
* The developer assigned to an issue might change.
* Fixes suggested by developers might be subject to _quality assurance_, to ensure that they
    * fix the error under all circumstances
    * do not introduce new errors
* Issues may be _reopened_ if the fix is inadequate or new information becomes available.

With this in mind, the states an issue report can go through become a bit more complex. Note how issues start in a state of "UNCONFIRMED" as long as nobody has assessed them, and how issues are marked as "CLOSED" (the final state) only after the resolution has been checked by quality assurance.

In [ ]:
# ignore

life_cycle.node('Unconfirmed', label="<<b>UNCONFIRMED</b>>", penwidth='2.0')
# life_cycle.node('Verified', label="<<b>VERIFIED</b>>")
life_cycle.node('Closed', label="<<b>CLOSED</b>>", penwidth='2.0')
life_cycle.node('Reopened', label="<<b>REOPENED</b>>", style='filled')
life_cycle.node('New', label="<<b>NEW</b>>", penwidth='1.0')

life_cycle.edge('Unconfirmed', 'New', label="Confirmed as \"new\"")
life_cycle.edge('Unconfirmed', 'Closed', label=r"Resolved\las \"invalid\"\lor \"duplicate\"")
life_cycle.edge('Assigned', 'New', label="Unassigned")
life_cycle.edge('Resolved', 'Closed', label=r"Quality Assurance\lconfirms fix")
life_cycle.edge('Resolved', 'Reopened', label=r"Quality Assurance\lnot satisfied")
life_cycle.edge('Reopened', 'Assigned', label=r"Assigned\lto developer")
# life_cycle.edge('Verified', 'Closed', label="Bug is closed")
life_cycle.edge('Closed', 'Reopened', label=r"Bug is\lreopened")

life_cycle

Such states (and the implied transitions between them) can be found in any issue tracking system. The actual names may vary (ASSIGNED is also known as IN_PROGRESS; instead of or besides CLOSED, one can also have VERIFIED), but their meanings are always the same. 

Whatever the states and resolutions are called (and they can be configured, anyway), the important thing is that they be used consistently for your project and in your organization. This is how at any point, any team member can see

* what the most pressing tasks are;
* what he or she should be working on; and
* what the overall state of the project is.

The latter point is particularly important, since an issue tracker need not only be used for tracking bugs, but also for tracking and assigning _features_ – notably features that have been decided to be part of the product. Hence, if you want to have some new feature implemented, you can mark it as a "NEW" issue, break it into subfeatures, also all marked as "NEW", and then assign the associated subtasks to individual developers. The feature would then be ready when all subtasks are completed as "FIXED".

This way, you can _organize an entire software project through the issue tracker._ Start with the first issue "The product is missing", and then keep on defining and assigning subtasks; when all are resolved, the product will be ready. Since some issue trackers (including _Redmine_) also allow developers to report how far they got with resolving an issue (say, "10%", "50%", or "90%"), you can even estimate how long resolving the open issues will take.

Over time, an issue database not only becomes an important tool for managing bug fixing and organizing software development, it also holds a trove of data about where, what, and why specific issues were addressed. In the [next chapter](ChangeCounter.ipynb), we will explore how to mine and evaluate such data.

In [ ]:
# ignore

# We're done, so we shut down Redmine and associated processes
redmine_process.terminate()
redmine_gui.close()

In [ ]:
# ignore
os.system("pkill ruby");

## Synopsis

This chapter provides no functionality that could be used by third-party code.

## Lessons Learned

* In an organization, fixing bugs and issues is best organized through an _issue tracker_.
* An _issue tracker_ allows issue reports
    * to be _collected_;
    * to assign them with a _state_ (from NEW to CLOSED) in its _life cycle_;
    * to be _assigned_ to developers; and
    * to assign them with a _resolution_ (including FIXED, DUPLICATE, and WORKSFORME).
* Issue trackers can organize the entire software development, including new products and features.

## Next Steps

* The [chapter on mining version histories](ChangeCounter.ipynb) describes how to _mine_ and _analyze_ the data from bug and version databases.

## Background

The survey by Bettenburg et al. \cite{Bettenburg2008} is the seminal work on how developers make use of bug reports. The study by Bertram et al. \cite{Bertram2010} highlights how issue trackers are not just databases, but "a focal point for communication and coordination for many stakeholders within and beyond the software team."

The report by \cite{Glerum2009} provides details on how _Windows Error Reporting_ automates the processing of error reports coming from an installed base of a billion machines, providing impressive numbers on bugs in the wild.

Research on bug databases has very much focused on how to make predictions such as automatically identifying duplicates \cite{Wang2008}, possible bug locations \cite{Kim2013}, or automatically assigning issues to developers \cite{Anvik2006}. Herzig et al.  \cite{Herzig2013}, however point out that much of the information in bug database is not classified properly enough, for instance to reliably distinguish between bugs and features.

Mozilla's [Bugzilla bug database](https://bugzilla.mozilla.org/) is one of the largest publicly available issue databases. It is worth browsing for real-world bug reports, and also contains important articles on how to write and use bug reports.